# Исследование по метрикам на основе данных моего произношения

## Импорт библиотеки

In [1]:
# Для взаимодействия с системой
import sys
import os
import re
# Для работы с массивами чисел
import numpy as np 
# Для оценки качества распознавания речи
from jiwer import wer, cer, mer, wil
# Для вычисления расстояния Левенштейна
import Levenshtein
# Для статистического анализа данных
from scipy.stats import pearsonr, linregress
from statsmodels.tsa.stattools import adfuller
import pymannkendall as mk
# Для работы с базами данных PostgreSQL
import psycopg2
# Для работы с данными в формате DataFrame
import pandas as pd
# Для визуализации данных
import matplotlib.pyplot as plt
# Для работы со звуковыми файлами
import io
import soundfile as sf
import librosa
# Для обработки естественного языка
import nltk
import string
from nltk.stem import WordNetLemmatizer
# Для работы с различными сервисами распознавания речи
from AutomaticSpeechRecognitionYandex import AutomaticSpeechRecognitionYandex
from AutomaticSpeechRecognitionSaluteSpeech import AutomaticSpeechRecognitionSaluteSpeech
from AutomaticSpeechRecognitionMBART50 import AutomaticSpeechRecognitionMBART50

from tokens import FOLDER_ID, OAUTH, CLIENT_ID, CLIENT_SECRET

## Загрузка данных с БД

In [2]:
# Сбор список слов
from sentences_text import sentences_text

### Коллекция слов

In [3]:
for theme, words in sentences_text.items():
    print(f"Тема: {theme}")
    print("Предложение:", words)
    print("\n")

Тема: Искусственный интеллект
Предложение: Искусственный интеллект (ИИ) - это область компьютерных наук, создающая системы для выполнения задач, требующих человеческого интеллекта: распознавание речи, обработка изображений, принятие решений. Искусственный интеллект проникает во все сферы жизни, от рекомендаций в онлайн-магазинах до медицинских диагнозов.

Главное преимущество ИИ - способность обрабатывать большие объемы данных и выявлять закономерности. В медицине он помогает диагностировать заболевания, анализируя снимки и историю болезни. В бизнесе искусственный интеллект используется для прогнозирования рынка, оптимизации цепочек поставок и автоматизации процессов.

ИИ базируется на алгоритмах машинного обучения, позволяющих системам обучаться на основе данных. Современные ИИ-системы используют глубокие нейронные сети, имитирующие работу мозга. Для разработки таких моделей требуются мощные вычислительные ресурсы и инструменты, такие как TensorFlow и PyTorch.

Искусственный интеллект

### Подключение к БД

In [4]:
%%time
# Соединение с БД
conn = psycopg2.connect(
    host="localhost",
    port="5432",
    database="asr_text",
    user="postgres",
    password="postgres"
)
cur = conn.cursor()

CPU times: user 7.96 ms, sys: 1.96 ms, total: 9.92 ms
Wall time: 16.1 ms


In [5]:
%%time
# Получение данных из таблицы
cur.execute("SELECT * FROM audio_records")
data = cur.fetchall()

CPU times: user 252 ms, sys: 340 ms, total: 592 ms
Wall time: 744 ms


In [6]:
%%time
# Закрытие соединения
cur.close()
conn.close()

CPU times: user 0 ns, sys: 830 µs, total: 830 µs
Wall time: 589 µs


### Создание и анализ датафрейма

In [7]:
# Создание DataFrame из полученных данных
df = pd.DataFrame(data, columns=['id', 'topic', 'paragraph_text', 'transcript_text_yandex', 'transcript_text_salutespeech', 'transcript_text_mbart50', 'voice_recording', 'record_date'])

In [8]:
# Преобразование столбца record_date в формат datetime
df['record_date'] = pd.to_datetime(df['record_date'])

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 461 entries, 0 to 460
Data columns (total 8 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   id                            461 non-null    int64         
 1   topic                         461 non-null    object        
 2   paragraph_text                461 non-null    object        
 3   transcript_text_yandex        461 non-null    object        
 4   transcript_text_salutespeech  461 non-null    object        
 5   transcript_text_mbart50       461 non-null    object        
 6   voice_recording               461 non-null    object        
 7   record_date                   461 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(6)
memory usage: 28.9+ KB


In [12]:
df.head()

,id,topic,paragraph_text,transcript_text_yandex,transcript_text_salutespeech,transcript_text_mbart50,voice_recording,record_date
0,5,Искусственный интеллект,Искусственный интеллект - революционная технол...,Рвотчины технари мнящие мир и проносящие на че...,"Эффект революционный техногии, меняющий мир, п...","они киллект, революция, тихонги, мне очень яны...","[b'O', b'g', b'g', b'S', b'\x00', b'\x02', b'\...",2024-06-15
1,13,Веб-сокет,Веб-сокеты - мощный инструмент для разработки ...,Веб секрет мощный интернет для разработки веб ...,Рен постояние 3 направлений сидни и низкая за...,"ведь все-таки, тёмненький интеллигент, проживш...","[b'O', b'g', b'g', b'S', b'\x00', b'\x02', b'\...",2024-06-16
2,14,Плавание,"Плавание - это вид спорта и активного отдыха, ...",Прямо на этой битве спорят и активисты выходят...,"Про мне это вид спорт и активный. Выходи, закл...","про мне эти видосы, и активные охоты и закруча...","[b'O', b'g', b'g', b'S', b'\x00', b'\x02', b'\...",2024-06-16
3,23,Технологическая сингулярность,"Технологическая сингулярность - это теория, пр...",Процессорывающий момент когда интеллект провод...,Проводит человеческий интеллект и технологии ...,"— из-вторых, неверность этот яры, протягивающи...","[b'O', b'g', b'g', b'S', b'\x00', b'\x02', b'\...",2024-06-16
4,38,Плавание,Существует несколько стилей плавания с разными...,Как ровными техническими осипшими крыла на гру...,"Считывание, сколько стилей прямо сравнимы техн...","— ты говорю, нет, как ощутили, я промолленными...","[b'O', b'g', b'g', b'S', b'\x00', b'\x02', b'\...",2024-06-17


In [13]:
# Экспорт DataFrame в CSV файл
df.to_csv('database.csv', index=False)

## Предобработка данных

##### Поскольку модель выдавала текст с знаками и другие, поэтому будем устранять их и заново рассчитать метрику

### Группировка по датам и ID

In [14]:
# pd.set_option('display.max_colwidth', None)  # Устанавливаем неограниченную ширину колонок
# pd.set_option('display.max_columns', None)   # Устанавливаем вывод всех колонок
# pd.set_option('display.max_rows', None)      # Устанавливаем вывод всех строк

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 461 entries, 0 to 460
Data columns (total 8 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   id                            461 non-null    int64         
 1   topic                         461 non-null    object        
 2   paragraph_text                461 non-null    object        
 3   transcript_text_yandex        461 non-null    object        
 4   transcript_text_salutespeech  461 non-null    object        
 5   transcript_text_mbart50       461 non-null    object        
 6   voice_recording               461 non-null    object        
 7   record_date                   461 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(6)
memory usage: 28.9+ KB


In [16]:
# Сбросим datetime чтобы сделать группировку, иначе не получим из за уникальности
df['record_date'] = df['record_date'].dt.date

In [17]:
# Сгруппируем по датам и ID
df = df.groupby(['record_date', 'id']).sum()

In [18]:
display(df.head(10))

topic  \
record_date id                            
2024-06-15  1   Искусственный интеллект   
            2   Искусственный интеллект   
            4   Искусственный интеллект   
            5   Искусственный интеллект   
2024-06-16  6   Искусственный интеллект   
            7   Искусственный интеллект   
            8   Искусственный интеллект   
            9                 Веб-сокет   
            10                Веб-сокет   
            12                Веб-сокет   

                                                   paragraph_text  \
record_date id                                                      
2024-06-15  1   Искусственный интеллект (ИИ) - это область ком...   
            2   Главное преимущество ИИ - способность обрабаты...   
            4   ИИ базируется на алгоритмах машинного обучения...   
            5   Искусственный интеллект - революционная технол...   
2024-06-16  6   Искусственный интеллект (ИИ) - это область ком...   
            7   Главное преимущество ИИ - способность обрабаты...   
            8   ИИ базируется на алгоритмах машинного обучения...   
            9   Веб-сокеты - это технология для обмена данными...   
            10  Главное преимущество веб-сокетов - поддержка п...   
            12  Веб-сокеты работают поверх протокола TCP и исп...   

                                           transcript_text_yandex  \
record_date id                                                      
2024-06-15  1   Искусственный интеллект это вопрос компьютерны...   
            2   Структурный тракт Трапающий тракт Так из за по...   
            4   Секс интеллект позируется на алгоритмах машина...   
            5   Рвотчины технари мнящие мир и проносящие на че...   
2024-06-16  6   Интеллект Видать трепающий челябинского интелл...   
            7   Степощепанник обрабатывает пращи о примыканиях...   
            8   Исключительный интеллект революционные техноло...   
            9   Технология Мы не станем выраженным равномерным...   
            10  Го мне прымщ тебя высек эта поддержка штейна в...   
            12  Веб щеки для работы поверх протокола как то се...   

                                     transcript_text_salutespeech  \
record_date id                                                      
2024-06-15  1                                                       
            2   Осуществлённый тракт от оплаты компьютерных но...   
            4   Интеллект позируется на алгоритмах машины появ...   
            5   Эффект революционный техногии, меняющий мир, п...   
2024-06-16  6   Интеллект компьютерных наук, состоящих материа...   
            7                                                       
            8   Искусственный интеллект, революционные техноло...   
            9    В отличие от этой перезапросом и всякой обесп...   
            10  Да, мне пришло вот это поддержка почтенного св...   
            12  Vip все какие-то работы поверх протокола, как,...   

                                          transcript_text_mbart50  \
record_date id                                                      
2024-06-15  1   едивствительный характер, а то я, попросту, ко...   
            2   здесь копустимный тракт, а то просит купечески...   
            4   здесь к примеру интеллигент, пояснивая причину...   
            5   они киллект, революция, тихонги, мне очень яны...   
2024-06-16  6   безопасный интеллект, дай-капитатурных наок, р...   
            7   голубнеж тунеядцы, стипендиаты, стипендистки, ...   
            8   здесь к счастью, интеллигенты, ловящие на них ...   
            9   ибо, посяки эти, как ты нравы, мы не таны, мы ...   
            10  голубня, прыгнув тебе, джигурка, это поддержка...   
            12  -- беспокоит я до обед, про такое коте-сепье и...   

                                                  voice_recording  
record_date id                                                     
2024-06-15  1   [b'O', b'g', b'g', b'S', b'\x00', b'\x02', b'\...

In [19]:
# Считаем количество записей и сбрасываем индекс для наглядности
id_count_per_date = df.groupby('record_date').size().reset_index(name='count')

print(id_count_per_date)

   record_date  count
0   2024-06-15      4
1   2024-06-16     19
2   2024-06-17     40
3   2024-06-18     40
4   2024-06-26     39
5   2024-06-27     40
6   2024-06-30     20
7   2024-07-01     19
8   2024-07-16     20
9   2024-07-17     20
10  2024-07-19     20
11  2024-07-21     20
12  2024-07-23     20
13  2024-07-25     20
14  2024-07-27     20
15  2024-08-01     20
16  2024-08-04     20
17  2024-08-09     20
18  2024-08-15     20
19  2024-08-20     20


Теперь количество записей в каждой дате совпадает, значит правильно сгруппировано, теперь переходим дальше к следующему этапу

### Восстановление пустых значений

##### Некоторые значения в столбцах и колонках распознанный текст пуст, нужно восстановить его через записанное аудиофайл и по API внешного сервиса 

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 461 entries, (datetime.date(2024, 6, 15), 1) to (datetime.date(2024, 8, 20), 476)
Data columns (total 6 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   topic                         461 non-null    object
 1   paragraph_text                461 non-null    object
 2   transcript_text_yandex        461 non-null    object
 3   transcript_text_salutespeech  461 non-null    object
 4   transcript_text_mbart50       461 non-null    object
 5   voice_recording               461 non-null    object
dtypes: object(6)
memory usage: 26.8+ KB


#### Класс для обработки каждой строки

In [36]:
class TranscriptUpdater:
    def __init__(self, df, yandex_asr, salute_asr, mbart_asr):
        self.df = df
        self.yandex_asr = yandex_asr
        self.salute_asr = salute_asr
        self.mbart_asr = mbart_asr

    def is_invalid_transcript(self, transcript):
        """Проверяет, содержит ли текст ошибки или является пустым."""
        return pd.isna(transcript) or transcript.strip() == "" or "Ошибка" in transcript or "HTTP Error" in transcript

    def update_transcripts(self):
        """Перезаписывает транскрипции для строк с ошибками или пустыми значениями."""
        for index, row in self.df.iterrows():

            # Проверяем колонку Yandex
            if self.is_invalid_transcript(row['transcript_text_yandex']):
                print(f"Перезапись Yandex для ID: {index}")
                audio_data = self.load_audio_from_path(row['voice_recording']) # Получаем данные аудио из строки
                if audio_data is not None:
                    old_transcript_yandex = row['transcript_text_yandex']
                    try:
                        audio_data = self.trim_audio(audio_data) # Обрезаем до 30 секунд
                        new_transcript_yandex = self.yandex_asr.recognize_speech_audio_data(audio_data, print_result=False)
                        if is_invalid_transcript(new_transcript_yandex):
                            self.df.at[index, 'transcript_text_yandex'] = None
                            print(f"Не удалось распознать текст Yandex для ID: {index}. Установлено значение None.")
                        else:
                            self.df.at[index, 'transcript_text_yandex'] = new_transcript_yandex
                            print(f"Исправлено Yandex для ID: {index}. Старый текст: {old_transcript_yandex}, Новый текст: {new_transcript_yandex}")
                    except Exception as e:
                        print(f"Ошибка при распознавании Yandex для ID: {index}: {e}")
                        # Присваиваем None в случае ошибки
                        self.df.at[index, 'transcript_text_yandex'] = None
                else:
                    print(f"Ошибка: отсутствует аудиофайл для Yandex ID: {index}")

            # Проверяем колонку SaluteSpeech
            if self.is_invalid_transcript(row['transcript_text_salutespeech']):
                print(f"Перезапись SaluteSpeech для ID: {index}")
                audio_data = self.load_audio_from_path(row['voice_recording']) # Получаем данные аудио из строки
                if audio_data is not None:
                    old_transcript_salutespeech = row['transcript_text_salutespeech']
                    try:
                        audio_data = self.trim_audio(audio_data) # Обрезаем до 30 секунд
                        new_transcript_salutespeech = self.salute_asr.recognize_speech_salute(audio_data, print_result=False)
                        if is_invalid_transcript(new_transcript_yandex):
                            self.df.at[index, 'transcript_text_salutespeech'] = None
                            print(f"Не удалось распознать текст SaluteSpeech для ID: {index}. Установлено значение None.")
                        else:
                            self.df.at[index, 'transcript_text_salutespeech'] = new_transcript_salutespeech
                            print(f"Исправлено SaluteSpeech для ID: {index}. Старый текст: {old_transcript_salutespeech}, Новый текст: {new_transcript_salutespeech}")
                    except Exception as e:
                        print(f"Ошибка при распознавании SaluteSpeech для ID: {index}: {e}")
                        # Присваиваем None в случае ошибки
                        self.df.at[index, 'transcript_text_salutespeech'] = None
                else:
                    print(f"Ошибка: отсутствует аудиофайл для SaluteSpeech ID: {index}")

            # Проверяем колонку MBART50
            if self.is_invalid_transcript(row['transcript_text_mbart50']):
                print(f"Перезапись MBART50 для ID: {index}")
                audio_data = self.load_audio_from_path(row['voice_recording']) # Получаем данные аудио из строки
                if audio_data is not None:
                    old_transcript_mbart50 = row['transcript_text_mbart50']
                    try:
                        audio_data = self.trim_audio(audio_data) # Обрезаем до 30 секунд
                        new_transcript_mbart50 = self.mbart_asr.transcribe_audio(audio_data, print_result=False)
                        self.df.at[index, 'transcript_text_mbart50'] = new_transcript_mbart50
                        print(f"Исправлено MBART50 для ID: {index}. Старый текст: {old_transcript_mbart50}, Новый текст: {new_transcript_mbart50}")
                    except Exception as e:
                        print(f"Ошибка при распознавании SaluteSpeech для ID: {index}: {e}")
                        # Присваиваем None в случае ошибки
                        self.df.at[index, 'transcript_text_mbart50'] = None
                else:
                    print(f"Ошибка: отсутствует аудиофайл для MBART50 ID: {index}")


    def load_audio_from_path(self, audio_data):
        """Загружает аудиоданные из байтового массива."""
        try:
            # Проверяем, что аудиоданные корректного размера
            if len(audio_data) % 4 != 0:
                print(f"Размер аудиоданных ({len(audio_data)}) не кратен размеру элемента (4). Обрезка...")
                # Обрезаем данные до ближайшего значения, кратного 4
                audio_data = audio_data[:len(audio_data) - (len(audio_data) % 4)]
    
            audio_array = np.frombuffer(audio_data, dtype=np.int16)  # Предполагая, что данные в int16
            sample_rate = 16000
            
            # Проверьте, нужно ли ресэмплинг
            if sample_rate != 16000:
                print("Частота дискретизации аудио должна быть 16000 Гц. Изменен аудиофайл на 16000 Гц.")
                audio_array = librosa.resample(audio_array, orig_sr=sample_rate, target_sr=16000)
            
            return audio_array
        except Exception as e:
            print(f"Ошибка загрузки аудио файла: {e}")
            return None

    def trim_audio(self, audio_data, max_seconds=30):
        """Обрезает аудио до заданной длины."""
        sample_rate = 16000
        max_samples = int(max_seconds * sample_rate)
        if len(audio_data) > max_samples:
            print(f"Аудио длиннее {max_seconds} секунд ({len(audio_data)} сэмплов), обрезаем до {max_samples} сэмплов.")
            return audio_data[:max_samples]
        else:
            print(f"Аудио короче {max_seconds} секунд ({len(audio_data)} сэмплов), обрезка не требуется.")
            return audio_data

#### Инициализация класса

In [37]:
%%time
yandex_asr = AutomaticSpeechRecognitionYandex(FOLDER_ID, OAUTH)
salute_asr = AutomaticSpeechRecognitionSaluteSpeech(CLIENT_ID, CLIENT_SECRET)
# mbart_asr = AutomaticSpeechRecognitionMBART50()

CPU times: user 117 ms, sys: 1.93 ms, total: 119 ms
Wall time: 568 ms


#### Предобработка

In [38]:
%%time
# Обновление транскрипций в датасете
transcript_updater = TranscriptUpdater(df, yandex_asr, salute_asr, mbart_asr=None)
transcript_updater.update_transcripts()

Перезапись SaluteSpeech для ID: (datetime.date(2024, 6, 15), 1)
Аудио короче 30 секунд (422912 сэмплов), обрезка не требуется.
Ошибка при распознавании SaluteSpeech для ID: (datetime.date(2024, 6, 15), 1): The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()
Перезапись SaluteSpeech для ID: (datetime.date(2024, 6, 16), 7)
Аудио короче 30 секунд (451584 сэмплов), обрезка не требуется.
Ошибка при распознавании SaluteSpeech для ID: (datetime.date(2024, 6, 16), 7): The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()
Перезапись Yandex для ID: (datetime.date(2024, 6, 26), 138)
Аудио длиннее 30 секунд (480256 сэмплов), обрезаем до 480000 сэмплов.
Ошибка при распознавании Yandex для ID: (datetime.date(2024, 6, 26), 138): The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()
Перезапись SaluteSpeech для ID: (datetime.date(2024, 6, 26), 150)
Аудио короче 30 секунд (384000 сэмплов), 

In [22]:
display(transcript_updater.df.head(10))

topic  \
record_date id                            
2024-06-15  1   Искусственный интеллект   
            2   Искусственный интеллект   
            4   Искусственный интеллект   
            5   Искусственный интеллект   
2024-06-16  6   Искусственный интеллект   
            7   Искусственный интеллект   
            8   Искусственный интеллект   
            9                 Веб-сокет   
            10                Веб-сокет   
            12                Веб-сокет   

                                                   paragraph_text  \
record_date id                                                      
2024-06-15  1   Искусственный интеллект (ИИ) - это область ком...   
            2   Главное преимущество ИИ - способность обрабаты...   
            4   ИИ базируется на алгоритмах машинного обучения...   
            5   Искусственный интеллект - революционная технол...   
2024-06-16  6   Искусственный интеллект (ИИ) - это область ком...   
            7   Главное преимущество ИИ - способность обрабаты...   
            8   ИИ базируется на алгоритмах машинного обучения...   
            9   Веб-сокеты - это технология для обмена данными...   
            10  Главное преимущество веб-сокетов - поддержка п...   
            12  Веб-сокеты работают поверх протокола TCP и исп...   

                                           transcript_text_yandex  \
record_date id                                                      
2024-06-15  1   Искусственный интеллект это вопрос компьютерны...   
            2   Структурный тракт Трапающий тракт Так из за по...   
            4   Секс интеллект позируется на алгоритмах машина...   
            5   Рвотчины технари мнящие мир и проносящие на че...   
2024-06-16  6   Интеллект Видать трепающий челябинского интелл...   
            7   Степощепанник обрабатывает пращи о примыканиях...   
            8   Исключительный интеллект революционные техноло...   
            9   Технология Мы не станем выраженным равномерным...   
            10  Го мне прымщ тебя высек эта поддержка штейна в...   
            12  Веб щеки для работы поверх протокола как то се...   

                                     transcript_text_salutespeech  \
record_date id                                                      
2024-06-15  1                                                None   
            2   Осуществлённый тракт от оплаты компьютерных но...   
            4   Интеллект позируется на алгоритмах машины появ...   
            5   Эффект революционный техногии, меняющий мир, п...   
2024-06-16  6   Интеллект компьютерных наук, состоящих материа...   
            7                                                None   
            8   Искусственный интеллект, революционные техноло...   
            9    В отличие от этой перезапросом и всякой обесп...   
            10  Да, мне пришло вот это поддержка почтенного св...   
            12  Vip все какие-то работы поверх протокола, как,...   

                                          transcript_text_mbart50  \
record_date id                                                      
2024-06-15  1   едивствительный характер, а то я, попросту, ко...   
            2   здесь копустимный тракт, а то просит купечески...   
            4   здесь к примеру интеллигент, пояснивая причину...   
            5   они киллект, революция, тихонги, мне очень яны...   
2024-06-16  6   безопасный интеллект, дай-капитатурных наок, р...   
            7   голубнеж тунеядцы, стипендиаты, стипендистки, ...   
            8   здесь к счастью, интеллигенты, ловящие на них ...   
            9   ибо, посяки эти, как ты нравы, мы не таны, мы ...   
            10  голубня, прыгнув тебе, джигурка, это поддержка...   
            12  -- беспокоит я до обед, про такое коте-сепье и...   

                                                  voice_recording  
record_date id                                                     
2024-06-15  1   [b'O', b'g', b'g', b'S', b'\x00', b'\x02', b'\...

#### Сохраним после изменения

In [23]:
# Экспорт DataFrame в CSV файл
df.to_csv('database_preprocessing.csv', index=True)

### Лемматизация текста

#### Загрузка необходимых ресурсов NLTK

In [24]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /home/redalexdad/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/redalexdad/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [25]:
lemmatizer = WordNetLemmatizer()

#### Функция для лемматизации текста


In [26]:
def lemmatize_text(text):
    # Приведение к нижнему регистру
    text = text.lower()
    # Удаление знаков препинания
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Токенизация текста
    words = nltk.word_tokenize(text)
    # Лемматизация каждого слова
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    # Возвращаем лемматизированный текст
    return ' '.join(lemmatized_words)

In [27]:
# Применяем лемматизацию к нужной колонке (например, к 'paragraph_text')
df['lemmatized_text'] = df['paragraph_text'].apply(lemmatize_text)

In [28]:
df[['paragraph_text', 'lemmatized_text']].head(5)

paragraph_text  \
record_date id                                                      
2024-06-15  1   Искусственный интеллект (ИИ) - это область ком...   
            2   Главное преимущество ИИ - способность обрабаты...   
            4   ИИ базируется на алгоритмах машинного обучения...   
            5   Искусственный интеллект - революционная технол...   
2024-06-16  6   Искусственный интеллект (ИИ) - это область ком...   

                                                  lemmatized_text  
record_date id                                                     
2024-06-15  1   искусственный интеллект ии это область компьют...  
            2   главное преимущество ии способность обрабатыва...  
            4   ии базируется на алгоритмах машинного обучения...  
            5   искусственный интеллект революционная технолог...  
2024-06-16  6   искусственный интеллект ии это область компьют...

In [29]:
for i in range(5):
    print(df['lemmatized_text'][i])
    print('-'*10)
    print(df['transcript_text_yandex'][i])
    print('='*10)

искусственный интеллект ии это область компьютерных наук создающая системы для выполнения задач требующих человеческого интеллекта распознавание речи обработка изображений принятие решений искусственный интеллект проникает во все сферы жизни от рекомендаций в онлайнмагазинах до медицинских диагнозов
----------
Искусственный интеллект это вопрос компьютерных наук Проникать Банан в магазинах
главное преимущество ии способность обрабатывать большие объемы данных и выявлять закономерности в медицине он помогает диагностировать заболевания анализируя снимки и историю болезни в бизнесе искусственный интеллект используется для прогнозирования рынка оптимизации цепочек поставок и автоматизации процессов
----------
Структурный тракт Трапающий тракт Так из за поражений Проникать во все
ии базируется на алгоритмах машинного обучения позволяющих системам обучаться на основе данных современные иисистемы используют глубокие нейронные сети имитирующие работу мозга для разработки таких моделей требуют

/tmp/ipykernel_88746/966112451.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(df['lemmatized_text'][i])
/tmp/ipykernel_88746/966112451.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(df['transcript_text_yandex'][i])


### Расчет метрики

#### Функция для расчета метрик по текстам (WER, CER, MER, WIL), включая расстояние Левенштейна

In [35]:
def calculate_metrics(reference, recognized, value_round=3):
    """
    Вычисляет метрики для распознанного текста с точностью до сотых, включая расстояние Левенштейна.
    
    Args:
    reference (str): Эталонный текст.
    recognized (str): Распознанный текст.
    
    Returns:
        dict: Словарь с метриками WER, CER, MER, WIL, округлёнными до двух знаков.
    """
    if pd.isna(recognized) or pd.isna(reference):
        return {
            'WER': None,
            'CER': None,
            'MER': None,
            'WIL': None,
            'Levenshtein': None
        }
        
    levenshtein_distance = Levenshtein.distance(reference, recognized)
    
    return {
        'WER': round(wer(reference, recognized), value_round),
        'CER': round(cer(reference, recognized), value_round),
        'MER': round(mer(reference, recognized), value_round),
        'WIL': round(wil(reference, recognized), value_round),
        'Levenshtein': levenshtein_distance
    }

#### Функция для расчета метрик по трем колонкам транскрипций

In [36]:
def calculate_all_metrics(row):
    """
    Рассчитывает метрики WER, CER, MER, WIL и расстояние Левенштейна для всех трех транскрипций.

    Args:
    row (pd.Series): Строка датафрейма, содержащая эталонный и распознанные тексты.

    Returns:
    pd.Series: Метрики для всех транскрипций.
    """
    reference_text = row["paragraph_text"]
    
    # Метрики для каждой из трех транскрипций
    metrics_yandex = calculate_metrics(reference_text, row["transcript_text_yandex"])
    metrics_salutespeech = calculate_metrics(reference_text, row["transcript_text_salutespeech"])
    metrics_mbart50 = calculate_metrics(reference_text, row["transcript_text_mbart50"])
    
    return pd.Series({
        # Метрики для Yandex
        'WER_yandex': metrics_yandex['WER'],
        'CER_yandex': metrics_yandex['CER'],
        'MER_yandex': metrics_yandex['MER'],
        'WIL_yandex': metrics_yandex['WIL'],
        'Levenshtein_yandex': metrics_yandex['Levenshtein'],
        
        # Метрики для SaluteSpeech
        'WER_salutespeech': metrics_salutespeech['WER'],
        'CER_salutespeech': metrics_salutespeech['CER'],
        'MER_salutespeech': metrics_salutespeech['MER'],
        'WIL_salutespeech': metrics_salutespeech['WIL'],
        'Levenshtein_salutespeech': metrics_salutespeech['Levenshtein'],
        
        # Метрики для MBART50
        'WER_mbart50': metrics_mbart50['WER'],
        'CER_mbart50': metrics_mbart50['CER'],
        'MER_mbart50': metrics_mbart50['MER'],
        'WIL_mbart50': metrics_mbart50['WIL'],
        'Levenshtein_mbart50': metrics_mbart50['Levenshtein']
    })

In [37]:
%%time
# Применяем функцию ко всему датафрейму
df_metrics = df.apply(calculate_all_metrics, axis=1)

CPU times: user 658 ms, sys: 4.78 ms, total: 662 ms
Wall time: 660 ms


In [38]:
%%time
# Объединяем результат с оригинальным датафреймом
df = pd.concat([df, df_metrics], axis=1)

CPU times: user 1.26 ms, sys: 42 µs, total: 1.3 ms
Wall time: 1.07 ms


In [39]:
# Сохраняем результат в CSV
df.to_csv("database_preprocessing_metric.csv")

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 461 entries, (datetime.date(2024, 6, 15), 1) to (datetime.date(2024, 8, 20), 476)
Data columns (total 22 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   topic                         461 non-null    object 
 1   paragraph_text                461 non-null    object 
 2   transcript_text_yandex        453 non-null    object 
 3   transcript_text_salutespeech  431 non-null    object 
 4   transcript_text_mbart50       461 non-null    object 
 5   voice_recording               461 non-null    object 
 6   lemmatized_text               461 non-null    object 
 7   WER_yandex                    453 non-null    float64
 8   CER_yandex                    453 non-null    float64
 9   MER_yandex                    453 non-null    float64
 10  WIL_yandex                    453 non-null    float64
 11  Levenshtein_yandex            453 non-null    float64
 12  WER_

In [45]:
df.describe()

,WER_yandex,CER_yandex,MER_yandex,WIL_yandex,Levenshtein_yandex,WER_salutespeech,CER_salutespeech,MER_salutespeech,WIL_salutespeech,Levenshtein_salutespeech,WER_mbart50,CER_mbart50,MER_mbart50,WIL_mbart50,Levenshtein_mbart50
count,453.000000,453.000000,453.000000,453.000000,453.000000,431.000000,431.000000,431.000000,431.000000,431.000000,461.000000,461.000000,461.000000,461.000000,461.000000
mean,0.798976,0.466684,0.755269,0.909035,144.430464,0.721819,0.409473,0.710571,0.875810,128.139211,0.967633,0.691252,0.928573,0.990885,215.535792
std,0.129720,0.119979,0.111888,0.072165,37.238916,0.112876,0.148384,0.113882,0.075411,47.315508,0.074109,0.056834,0.046305,0.012192,28.849936
min,0.389000,0.114000,0.378000,0.603000,32.000000,0.429000,0.137000,0.429000,0.632000,42.000000,0.724000,0.441000,0.700000,0.873000,100.000000
25%,0.721000,0.384000,0.686000,0.878000,119.000000,0.650000,0.303000,0.631000,0.830000,93.000000,0.925000,0.659000,0.905000,0.988000,197.000000
50%,0.805000,0.464000,0.763000,0.925000,143.000000,0.730000,0.372000,0.714000,0.890000,119.000000,0.953000,0.690000,0.932000,0.995000,217.000000
75%,0.892000,0.540000,0.838000,0.963000,170.000000,0.791000,0.491500,0.784000,0.930000,157.000000,1.000000,0.719000,0.957000,0.998000,233.000000
max,1.250000,0.912000,1.000000,1.000000,273.000000,1.000000,0.988000,1.000000,1.000000,310.000000,1.368000,0.941000,1.000000,1.000000,326.000000


In [40]:
df[df['WER_yandex'] == df['WER_yandex'].min()]

,,topic,paragraph_text,transcript_text_yandex,transcript_text_salutespeech,transcript_text_mbart50,voice_recording,lemmatized_text,WER_yandex,CER_yandex,MER_yandex,...,WER_salutespeech,CER_salutespeech,MER_salutespeech,WIL_salutespeech,Levenshtein_salutespeech,WER_mbart50,CER_mbart50,MER_mbart50,WIL_mbart50,Levenshtein_mbart50
record_date,id,,,,,,,,,,,,,,,,,,,,,
2024-07-27,368,Квантовые вычисления,Квантовые вычисления представляют собой револю...,Квантовые вычисления представляют собой револю...,Квантовые вычисления представляет собой револю...,квантей бюллетеней представляются побытичней т...,"[b'O', b'g', b'g', b'S', b'\x00', b'\x02', b'\...",квантовые вычисления представляют собой револю...,0.389,0.114,0.378,...,0.444,0.157,0.444,0.663,45.0,0.833,0.537,0.833,0.963,151.0
